In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
TestXpecgen.py: Tests for  the `xpecgen` package.
"""

import unittest
import math

from fastcat import fastcat as fc

# s = fc.calculate_spectrum(100, 12, 3, 10, epsrel=0.5, monitor=None)

In [15]:
class XpecgenTest(unittest.TestCase):
#     def test_clone(self):
#         """Test the Spectrum clone method"""
#         s2 = s.clone()
#         # Check same values
#         self.assertEqual(list(s.x), list(s2.x))
#         self.assertEqual(list(s.y), list(s2.y))
#         self.assertEqual(list(s.discrete), list(s2.discrete))
#         # Check alteration does not alter both instances
#         s.x[0] = 10.0
#         s.y[0] = 10.0
#         s.discrete[0][0] = 10.0
#         self.assertNotEqual(s.x[0], s2.x[0])
#         self.assertNotEqual(s.y[0], s2.y[0])
#         self.assertNotEqual(s.discrete[0][0], s2.discrete[0][0])

    def test_get_continuous_function(self):
        """Test the get_continuous_function method"""
        f = s.get_continuous_function()
        for p in zip(s.x, s.y):
            self.assertAlmostEqual(p[1], f(p[0]))

    def test_norm_functions(self):
        """Test the get_norm and set_norm methods"""
        s2 = s.clone()
        for w in [lambda x: 1, lambda x: x, fc.get_fluence_to_dose()]:
            s2.set_norm(13.0, w)
            self.assertAlmostEqual(s2.get_norm(w), 13.0)

    def test_attenuate(self):
        """Test the attenuate method"""
        # Unit distance, unit energy-independent attenuation coefficient -> attenuation with a factor 1/e
        s2 = s.clone()
        s2.attenuate(1, lambda x: 1)
        # Check continuous component changed
        for p in zip(s.y, s2.y):
            self.assertAlmostEqual(p[0] / math.e, p[1])
        # Check discrete component change
        for p in zip(s.discrete, s2.discrete):
            self.assertAlmostEqual(p[0][1] / math.e, p[1][1])

    def test_HVL_values(self):
        """Test to reproduce the values in Table III of Med. Phys. 43, 4655 (2016)"""

        # Calculate the emission spectra
        num_div = 20  # Points in each spectrum (quick calculation)
        s50 = fc.calculate_spectrum(50, 12, 3, num_div, epsrel=0.5, monitor=None)
        s80 = fc.calculate_spectrum(80, 12, 3, num_div, epsrel=0.5, monitor=None)
        s100 = fc.calculate_spectrum(100, 12, 3, num_div, epsrel=0.5, monitor=None)

        # Attenuate them
        s50.attenuate(0.12, fc.get_mu(13))  # 1.2 mm of Al
        s50.attenuate(100, fc.get_mu("air"))  # 100 cm of Air
        s80.attenuate(0.12, fc.get_mu(13))  # 1.2 mm of Al
        s80.attenuate(100, fc.get_mu("air"))  # 100 cm of Air
        s100.attenuate(0.12, fc.get_mu(13))  # 1.2 mm of Al
        s100.attenuate(100, fc.get_mu("air"))  # 100 cm of Air

        # Functions to calculate HVL in the sense of dose in Al
        fluence_to_dose = fc.get_fluence_to_dose()
        mu_al = fc.get_mu(13)

        # HVL in mm
        hvl50 = 10 * s50.hvl(0.5, fluence_to_dose, mu_al)
        hvl80 = 10 * s80.hvl(0.5, fluence_to_dose, mu_al)
        hvl100 = 10 * s100.hvl(0.5, fluence_to_dose, mu_al)

        self.assertAlmostEqual(hvl100, 2.37, places=1)
        self.assertAlmostEqual(hvl80, 1.85, places=1)
        self.assertAlmostEqual(hvl50, 1.20, places=1)

    def test_linear_combination(self):
        """Test linear combinations with Spectrum instances"""
        s2 = s.clone()
        s3 = s2+s2
        s4 = s2*2
        # Check same values
        self.assertEqual(list(s3.x), list(s4.x))
        self.assertEqual(list(s3.y), list(s4.y))
        self.assertEqual(list(s3.discrete), list(s4.discrete))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

(19, 41, 41) 19
bowtie is off False
really doing it!


F

ratio is 1.0 number of photons 0.0867941122973121


.....E
ERROR: test_norm_functions (__main__.XpecgenTest)
Test the get_norm and set_norm methods
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-15-8ba10f5f102d>", line 27, in test_norm_functions
    s2.set_norm(13.0, w)
  File "/home/xcite/fastcat-0.0.1/fastcat/fastcat.py", line 1988, in set_norm
    norm = self.get_norm(weight=weight) / value
  File "/home/xcite/fastcat-0.0.1/fastcat/fastcat.py", line 1971, in get_norm
    y2 = list(map(lambda x, y: w(x) * y, self.x, self.y))
  File "/home/xcite/fastcat-0.0.1/fastcat/fastcat.py", line 1971, in <lambda>
    y2 = list(map(lambda x, y: w(x) * y, self.x, self.y))
  File "/home/xcite/anaconda3/lib/python3.7/site-packages/scipy/interpolate/polyint.py", line 74, in __call__
    y = self._evaluate(x)
  File "/home/xcite/anaconda3/lib/python3.7/site-packages/scipy/interpolate/interpolate.py", line 659, in _evaluate
    below_bounds, above_bounds = self._check_boun

In [4]:
import unittest
import math

from fastcat import fastcat as fc

import numpy as np

class FastcatTest(unittest.TestCase):
    def test_dose_al(self):
        """Test if the dose matches MC estimates"""
        # Intialize
        angles = np.linspace(0,2*np.pi,2)
        s_fc = fc.Spectrum()
        phantom = fc.Catphan_515()
        # Test the C_25
        s_fc.load('C_spectrum_25')
        kernel = fc.Kernel(s_fc, 'CuGOS-784-micrometer')
        dose = phantom.return_projs(kernel,s_fc,angles,nphoton=2e7,
                                    mgy=0,return_dose=True)
        # C_25 test
        self.assertAlmostEqual(dose[-1], 0.04099205, places=2)
        
        # Test the C_25
        s_fc.load('W_spectrum_6')
        kernel = fc.Kernel(s_fc, 'CuGOS-784-micrometer')
        dose = phantom.return_projs(kernel,s_fc,angles,nphoton=2e7,
                                    mgy=0,return_dose=True)
        # C_25 test
        self.assertAlmostEqual(dose[-1], 0.1935003, places=2)
#         self.assertEquals(list(s.x), list(s2.x))
#         self.assertEqual(list(s.y), list(s2.y))
#         self.assertEqual(list(s.discrete), list(s2.discrete))
#         # Check alteration does not alter both instances
#         s.x[0] = 10.0
#         s.y[0] = 10.0
#         s.discrete[0][0] = 10.0
#         self.assertNotEqual(s.x[0], s2.x[0])
#         self.assertNotEqual(s.y[0], s2.y[0])
#         self.assertNotEqual(s.discrete[0][0], s2.discrete[0][0])

#     def test_get_continuous_function(self):
#         """Test the get_continuous_function method"""
#         f = s.get_continuous_function()
#         for p in zip(s.x, s.y):
#             self.assertAlmostEqual(p[1], f(p[0]))

#     def test_norm_functions(self):
#         """Test the get_norm and set_norm methods"""
#         s2 = s.clone()
#         for w in [lambda x: 1, lambda x: x, fc.get_fluence_to_dose()]:
#             s2.set_norm(13.0, w)
#             self.assertAlmostEqual(s2.get_norm(w), 13.0)

#     def test_attenuate(self):
#         """Test the attenuate method"""
#         # Unit distance, unit energy-independent attenuation coefficient -> attenuation with a factor 1/e
#         s2 = s.clone()
#         s2.attenuate(1, lambda x: 1)
#         # Check continuous component changed
#         for p in zip(s.y, s2.y):
#             self.assertAlmostEqual(p[0] / math.e, p[1])
#         # Check discrete component change
#         for p in zip(s.discrete, s2.discrete):
#             self.assertAlmostEqual(p[0][1] / math.e, p[1][1])

#     def test_HVL_values(self):
#         """Test to reproduce the values in Table III of Med. Phys. 43, 4655 (2016)"""

#         # Calculate the emission spectra
#         num_div = 20  # Points in each spectrum (quick calculation)
#         s50 = fc.calculate_spectrum(50, 12, 3, num_div, epsrel=0.5, monitor=None)
#         s80 = fc.calculate_spectrum(80, 12, 3, num_div, epsrel=0.5, monitor=None)
#         s100 = fc.calculate_spectrum(100, 12, 3, num_div, epsrel=0.5, monitor=None)

#         # Attenuate them
#         s50.attenuate(0.12, fc.get_mu(13))  # 1.2 mm of Al
#         s50.attenuate(100, fc.get_mu("air"))  # 100 cm of Air
#         s80.attenuate(0.12, fc.get_mu(13))  # 1.2 mm of Al
#         s80.attenuate(100, fc.get_mu("air"))  # 100 cm of Air
#         s100.attenuate(0.12, fc.get_mu(13))  # 1.2 mm of Al
#         s100.attenuate(100, fc.get_mu("air"))  # 100 cm of Air

#         # Functions to calculate HVL in the sense of dose in Al
#         fluence_to_dose = fc.get_fluence_to_dose()
#         mu_al = fc.get_mu(13)

#         # HVL in mm
#         hvl50 = 10 * s50.hvl(0.5, fluence_to_dose, mu_al)
#         hvl80 = 10 * s80.hvl(0.5, fluence_to_dose, mu_al)
#         hvl100 = 10 * s100.hvl(0.5, fluence_to_dose, mu_al)

#         self.assertAlmostEqual(hvl100, 2.37, places=1)
#         self.assertAlmostEqual(hvl80, 1.85, places=1)
#         self.assertAlmostEqual(hvl50, 1.20, places=1)

#     def test_linear_combination(self):
#         """Test linear combinations with Spectrum instances"""
#         s2 = s.clone()
#         s3 = s2+s2
#         s4 = s2*2
#         # Check same values
#         self.assertEqual(list(s3.x), list(s4.x))
#         self.assertEqual(list(s3.y), list(s4.y))
#         self.assertEqual(list(s3.discrete), list(s4.discrete))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 1.0 number of photons 0.04026543917603216
(19, 41, 41) 19
bowtie is off False
really doing it!


F

ratio is 1.0 number of photons 0.0053004634067323865



FAIL: test_dose_al (__main__.FastcatTest)
Test if the dose matches MC estimates
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-4-84deab6463dc>", line 29, in test_dose_al
    self.assertAlmostEqual(dose[-1], 0.1935003, places=2)
AssertionError: 0.19862507426368217 != 0.1935003 within 2 places (0.005124774263682186 difference)

----------------------------------------------------------------------
Ran 1 test in 3.591s

FAILED (failures=1)
